In [1]:
import pandas as pd
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from math import sqrt

In [2]:
pd.__version__

'1.4.2'

In [3]:
#Yellow Taxi Trip Records January2023:Training Data
df=pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet')

#Yellow Taxi Records February2023:Validattion Data
valid_df=pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet')

Answer to question 1

In [4]:
#Answer for question1
#total number of columns involved in Yellow Taxi Trip Records
len(df.columns)

19

In [5]:
#Function to convert duration into seconds
def convert_duration_to_minutes(duration):
    total_min=duration.total_seconds()/60
    return total_min

Answer to question2

In [6]:
#Answer to question2:Computing std()
df['duration']=df['tpep_dropoff_datetime']-df['tpep_pickup_datetime']
df['durationinmin']=df['duration'].apply(lambda x:convert_duration_to_minutes(x))
df['durationinmin'].std()

42.594351241920904

In [7]:
original_len=len(df.index)

Answer to question3

In [8]:
#Filtering df according to specified conditions
filtered_df=df.loc[(df.durationinmin>=1) & (df.durationinmin<=60)]

In [9]:
#Counting records in filtered dataframe
filtered_len=len(filtered_df.index)

In [10]:
#Fraction of records left
Fraction_left=filtered_len/original_len

In [11]:
#Fraction of records remaining
Fraction_left

0.9812202822125979

Answer to question4

In [12]:
filtered_df

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,...,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee,duration,durationinmin
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,N,161,141,2,...,1.00,0.5,0.00,0.0,1.0,14.30,2.5,0.00,0 days 00:08:26,8.433333
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,1.0,N,43,237,1,...,1.00,0.5,4.00,0.0,1.0,16.90,2.5,0.00,0 days 00:06:19,6.316667
2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.51,1.0,N,48,238,1,...,1.00,0.5,15.00,0.0,1.0,34.90,2.5,0.00,0 days 00:12:45,12.750000
3,1,2023-01-01 00:03:48,2023-01-01 00:13:25,0.0,1.90,1.0,N,138,7,1,...,7.25,0.5,0.00,0.0,1.0,20.85,0.0,1.25,0 days 00:09:37,9.616667
4,2,2023-01-01 00:10:29,2023-01-01 00:21:19,1.0,1.43,1.0,N,107,79,1,...,1.00,0.5,3.28,0.0,1.0,19.68,2.5,0.00,0 days 00:10:50,10.833333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3066761,2,2023-01-31 23:58:34,2023-02-01 00:12:33,NaN,3.05,NaN,None,107,48,0,...,0.00,0.5,3.96,0.0,1.0,23.76,NaN,NaN,0 days 00:13:59,13.983333
3066762,2,2023-01-31 23:31:09,2023-01-31 23:50:36,NaN,5.80,NaN,None,112,75,0,...,0.00,0.5,2.64,0.0,1.0,29.07,NaN,NaN,0 days 00:19:27,19.450000
3066763,2,2023-01-31 23:01:05,2023-01-31 23:25:36,NaN,4.67,NaN,None,114,239,0,...,0.00,0.5,5.32,0.0,1.0,26.93,NaN,NaN,0 days 00:24:31,24.516667
3066764,2,2023-01-31 23:40:00,2023-01-31 23:53:00,NaN,3.15,NaN,None,230,79,0,...,0.00,0.5,4.43,0.0,1.0,26.58,NaN,NaN,0 days 00:13:00,13.000000


In [13]:
#Setting up the columns
categorical=['PULocationID','DOLocationID']
numerical=['trip_distance']

In [14]:
#Cast to string for specified columns
filtered_df[categorical]=filtered_df[categorical].astype(str)

#Convert dataframe to dictionery
train_dict=filtered_df[categorical].to_dict(orient='records')

/tmp/ipykernel_96342/1823345594.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df[categorical]=filtered_df[categorical].astype(str)


In [15]:
train_dict

[{'PULocationID': '161', 'DOLocationID': '141'},
 {'PULocationID': '43', 'DOLocationID': '237'},
 {'PULocationID': '48', 'DOLocationID': '238'},
 {'PULocationID': '138', 'DOLocationID': '7'},
 {'PULocationID': '107', 'DOLocationID': '79'},
 {'PULocationID': '161', 'DOLocationID': '137'},
 {'PULocationID': '239', 'DOLocationID': '143'},
 {'PULocationID': '142', 'DOLocationID': '200'},
 {'PULocationID': '164', 'DOLocationID': '236'},
 {'PULocationID': '141', 'DOLocationID': '107'},
 {'PULocationID': '234', 'DOLocationID': '68'},
 {'PULocationID': '79', 'DOLocationID': '264'},
 {'PULocationID': '164', 'DOLocationID': '143'},
 {'PULocationID': '138', 'DOLocationID': '33'},
 {'PULocationID': '33', 'DOLocationID': '61'},
 {'PULocationID': '79', 'DOLocationID': '186'},
 {'PULocationID': '90', 'DOLocationID': '48'},
 {'PULocationID': '113', 'DOLocationID': '255'},
 {'PULocationID': '237', 'DOLocationID': '239'},
 {'PULocationID': '143', 'DOLocationID': '229'},
 {'PULocationID': '137', 'DOLocat

In [16]:
dv=DictVectorizer()

#Set up the dictionery vectorizer to obtain the feature_matrix
#Dimensionality of the resultant feature_matrix is shown below
X_train=dv.fit_transform(train_dict)
X_train.shape

(3009173, 515)

Answer to question5

In [17]:
#extracting the target variable
target='durationinmin'
y_train=filtered_df[target].values

In [18]:
#Training the LinearRegression model
lr=LinearRegression()
lr.fit(X_train,y_train)

LinearRegression()

In [19]:
#Making predictions
y_pred=lr.predict(X_train)
print(y_train.astype(float))

[ 8.43333333  6.31666667 12.75       ... 24.51666667 13.
 14.4       ]


In [20]:
print(y_train.astype(float).dtype)

float64


In [21]:
#Calculating the root mean_squared_error
sqrt(mean_squared_error(y_train,y_pred))

7.6492610279057605

Answer to question6

In [22]:
#Step1:Create the durationinmin column
valid_df['duration']=valid_df['tpep_dropoff_datetime']-valid_df['tpep_pickup_datetime']
valid_df['durationinmin']=valid_df['duration'].apply(lambda x:convert_duration_to_minutes(x))

#Step2:Filter the valid_df dataframe according to some conditions
filtered_valid_df=valid_df.loc[(valid_df.durationinmin>=1) & (valid_df.durationinmin<=60)]

#Step3:Cast to string for specified columns and convert Dataframe to Dictionery
filtered_valid_df[categorical]=filtered_valid_df[categorical].astype(str)
filtered_valid_dict=filtered_valid_df[categorical].to_dict(orient='records')

#Step4:Use the Dictionery Vectorizer to one hot encode valid_dict
X_valid=dv.transform(filtered_valid_dict)
X_valid.shape





/tmp/ipykernel_96342/2943660222.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_valid_df[categorical]=filtered_valid_df[categorical].astype(str)


(2855951, 515)

In [28]:
#Step5:Extract the target variable
target='durationinmin'
y_valid=filtered_valid_df[target].values

#Step6:Get predictions on valid data
y_pred_valid=lr.predict(X_valid)
y_pred_valid.shape

#Step7:Calculating the root mean_squared_error
sqrt(mean_squared_error(y_valid,y_pred_valid))


7.81183265470218